<a href="https://colab.research.google.com/github/rahultrada/Otto_Product_Classification/blob/main/xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import StratifiedKFold, train_test_split, StratifiedShuffleSplit, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
traindata = pd.read_csv('drive/My Drive/Colab Notebooks/Applied ML/train.csv')
testdata = pd.read_csv('drive/My Drive/Colab Notebooks/Applied ML/test.csv')

In [ ]:
trainys = traindata['target'].str[-1].astype(int) - 1 #convert 'class 1-9' to ints 0-8
trainxs = traindata.drop(['id', 'target'], axis=1)
testdata = testdata.drop(['id'], axis=1);

In [ ]:
X = trainxs.values
y = trainys.values
testx = testdata.values

In [ ]:
#try log preprocessing
Xlog = np.log1p(X)

In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(Xlog, y):
    x_train = Xlog[train_index]
    x_valid = Xlog[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]


In [ ]:
m1 = LogisticRegression(penalty='l1', solver='liblinear', n_jobs=-1, C=0.43, class_weight=None)
m2 = LogisticRegression(penalty='l2', solver='lbfgs', n_jobs=-1, C=0.01, class_weight=None, multi_class='multinomial')
for m in [m1,m2]:
  m.fit(x_train,y_train)
  print(log_loss(y_valid, m.predict_proba(x_valid)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.6637489799836348
0.6437697306216276


In [ ]:
#try sqrt preprocessing
Xsqrt = np.sqrt(X + 3.0/8)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(Xsqrt, y):
    x_train = Xsqrt[train_index]
    x_valid = Xsqrt[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]

In [ ]:
m1 = LogisticRegression(penalty='l1', solver='liblinear', n_jobs=-1, C=0.43, class_weight=None)
m2 = LogisticRegression(penalty='l2', solver='lbfgs', n_jobs=-1, C=0.01, class_weight=None, multi_class='multinomial')
for m in [m1,m2]:
  m.fit(x_train,y_train)
  print(log_loss(y_valid, m.predict_proba(x_valid)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.6533877454618872
0.6420435025134322


In [ ]:
#try no preprocessing
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(X, y):
    x_train = X[train_index]
    x_valid = X[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]
m1 = LogisticRegression(penalty='l1', solver='liblinear', n_jobs=-1, C=0.43, class_weight=None)
m2 = LogisticRegression(penalty='l2', solver='lbfgs', n_jobs=-1, C=0.01, class_weight=None, multi_class='multinomial')
for m in [m1,m2]:
  m.fit(x_train,y_train)
  print(log_loss(y_valid, m.predict_proba(x_valid)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.6586355869812888
0.6314476106275256


In [ ]:
#try standard scaling 
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(X, y):
    x_train = X[train_index]
    x_valid = X[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
m1 = LogisticRegression(penalty='l1', solver='liblinear', n_jobs=-1, C=0.43, class_weight=None)
m2 = LogisticRegression(penalty='l2', solver='lbfgs', n_jobs=-1, C=0.01, class_weight=None, multi_class='multinomial')
for m in [m1,m2]:
  m.fit(x_train_scaled,y_train)
  print(log_loss(y_valid, m.predict_proba(x_valid_scaled)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:1539: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 2.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


0.660594763862559
0.6527488790084442


In [ ]:
#try no preprocessing + gridsearch
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(X, y):
    x_train = X[train_index]
    x_valid = X[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]

# C = np.logspace(-4, 4, 10)
# multi_class = ['auto', 'multinomial']
# hyperparameters = dict(C=C, multi_class=multi_class)
# m = LogisticRegression(n_jobs=-1)
# clf = GridSearchCV(m, hyperparameters, scoring='neg_log_loss',cv=3, verbose=10)
# best_model = clf.fit(X, y)
# print(best_model.best_estimator_.get_params())

In [ ]:
#train best model from grid search on our split data
m = LogisticRegression(penalty='l2', solver='lbfgs', n_jobs=-1, C=0.0464, class_weight=None, multi_class='auto')
m.fit(x_train,y_train)
print(log_loss(y_valid, m.predict_proba(x_valid)))

0.6294769277036631


In [ ]:
np.savetxt('drive/My Drive/logregtrain.txt',m.predict_proba(x_train))
np.savetxt('drive/My Drive/logregvalid.txt',m.predict_proba(x_valid))
np.savetxt('drive/My Drive/logregtest.txt',m.predict_proba(testx))

In [ ]:
#now xgboost
import xgboost as xgb
#using log preprocessing
Xsqrt = np.sqrt(X+3.0/8)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(Xsqrt, y):
    x_train = Xsqrt[train_index]
    x_valid = Xsqrt[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]

In [ ]:
#SWITCH TO GPU!
m = xgb.XGBClassifier(learning_rate =0.1, n_estimators=525, max_depth=8, min_child_weight=3, subsample=0.7, 
                      colsample_bytree=0.7, nthread=4, seed=42, objective='multi:softprob',
                      tree_method='gpu_hist')
m = xgb.XGBClassifier(objective = 'multi:softprob', tree_method='gpu_hist')#rrr
m.fit(x_train, y_train)
print(log_loss(y_valid, m.predict_proba(x_valid)))

0.6413419997657951


In [ ]:
#try no preprocessing
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(X, y):
    x_train = X[train_index]
    x_valid = X[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]
m = xgb.XGBClassifier(learning_rate =0.1, n_estimators=525, max_depth=8, min_child_weight=3, subsample=0.7, 
                      colsample_bytree=0.7, nthread=4, seed=42, objective='multi:softprob',
                      tree_method='gpu_hist')
m = xgb.XGBClassifier(n_estimators=1200, learning_rate =0.04, max_depth=20, min_child_weight=2, gamma=0.88,
                      colsample_bytree=0.5, objective='multi:softprob', tree_method='gpu_hist')
#m.fit(x_train, y_train)
m.fit(X,y)
#print(log_loss(y_valid, m.predict_proba(x_valid)))
preds = m.predict_proba(testx)
#submission to kaggle, preds is output on test set
columns = ['Class_' + str(i + 1) for i in range(9)]
submission_df = pd.DataFrame(preds, columns=columns)
submission_df.index = submission_df.index + 1
submission_df.to_csv('submission.csv', sep=',', index_label='id')

In [ ]:
#try sqrt preprocessing
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
Xsqrt = np.sqrt(X+3.0/8)
for train_index, test_index in sss.split(Xsqrt, y):
    x_train = Xsqrt[train_index]
    x_valid = Xsqrt[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]
m = xgb.XGBClassifier(learning_rate =0.1, n_estimators=525, max_depth=8, min_child_weight=3, subsample=0.7, 
                      colsample_bytree=0.7, seed=42, objective='multi:softprob',
                      tree_method='gpu_hist')
m.fit(x_train, y_train)
print(log_loss(y_valid, m.predict_proba(x_valid)))

0.43857332353472084


In [ ]:
#randomized search
params = {'min_child_weight': [1, 3, 5, 10],
          'gamma': [0,1,2],
          'subsample': [0.7, 1.0],
          'colsample_bytree': [0.7, 1.0],
          'max_depth': [4,6,8],
          'n_estimators': [100,200,400,525,600]}
m = xgb.XGBClassifier(learning_rate = 0.1, seed=42, objective='multi:softprob', tree_method='gpu_hist')
clf = RandomizedSearchCV(m, params, scoring='neg_log_loss',cv=3, verbose=10, n_jobs=-1)
best_model = clf.fit(X, y)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   54.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 11.3min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 20.5min finished


In [ ]:
print(best_model.best_params_)

{'subsample': 0.7, 'n_estimators': 525, 'min_child_weight': 5, 'max_depth': 8, 'gamma': 0, 'colsample_bytree': 0.7}


In [ ]:
#run the (randomized search) best model on our split data
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(X, y):
    x_train = X[train_index]
    x_valid = X[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]
m = xgb.XGBClassifier(learning_rate =0.1, n_estimators=525, max_depth=8, min_child_weight=5, subsample=0.7, 
                      colsample_bytree=0.7, seed=42, objective='multi:softprob',
                      tree_method='gpu_hist')
m.fit(x_train,y_train)
print(log_loss(y_valid, m.predict_proba(x_valid)))

0.43889772881261285


In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(X, y):
    x_train = X[train_index]
    x_valid = X[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]
m = xgb.XGBClassifier(learning_rate =0.1, n_estimators=525, max_depth=8, min_child_weight=3, subsample=0.7, 
                      colsample_bytree=0.7, seed=42, objective='multi:softprob',
                      tree_method='gpu_hist')
m.fit(x_train, y_train, eval_metric='mlogloss', early_stopping_rounds=20, eval_set=[(x_valid,y_valid)])
print(log_loss(y_valid, m.predict_proba(x_valid)))
print(log_loss(y_train, m.predict_proba(x_train)))

[0]	validation_0-mlogloss:1.97533
Will train until validation_0-mlogloss hasn't improved in 20 rounds.
[1]	validation_0-mlogloss:1.81107
[2]	validation_0-mlogloss:1.68241
[3]	validation_0-mlogloss:1.57556
[4]	validation_0-mlogloss:1.4787
[5]	validation_0-mlogloss:1.39268
[6]	validation_0-mlogloss:1.32061
[7]	validation_0-mlogloss:1.25726
[8]	validation_0-mlogloss:1.20072
[9]	validation_0-mlogloss:1.14992
[10]	validation_0-mlogloss:1.10484
[11]	validation_0-mlogloss:1.06202
[12]	validation_0-mlogloss:1.02363
[13]	validation_0-mlogloss:0.988757
[14]	validation_0-mlogloss:0.959812
[15]	validation_0-mlogloss:0.932668
[16]	validation_0-mlogloss:0.90519
[17]	validation_0-mlogloss:0.881192
[18]	validation_0-mlogloss:0.85746
[19]	validation_0-mlogloss:0.836755
[20]	validation_0-mlogloss:0.816727
[21]	validation_0-mlogloss:0.799189
[22]	validation_0-mlogloss:0.782081
[23]	validation_0-mlogloss:0.766723
[24]	validation_0-mlogloss:0.751056
[25]	validation_0-mlogloss:0.738281
[26]	validation_0-mlo

In [ ]:

### concatenate outputs of first layer to create input to second layer


In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=0)
for train_index, test_index in sss.split(X, y):
    x_train = X[train_index]
    x_valid = X[test_index]
    y_train = y[train_index]
    y_valid = y[test_index]

In [ ]:
xgbtrain = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/xgbtrain.txt')
xgbvalid = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/xgbvalid.txt')
xgbtest = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/xgbtest.txt')

In [ ]:
print(xgbtrain.shape)
print(xgbvalid.shape)
print(xgbtest.shape)

(49502, 9)
(12376, 9)
(144368, 9)


In [ ]:
rftrain = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/RFTrain.txt')
rfvalid = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/RFVal.txt')
rftest = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/RFTest.txt')
assert(rftrain.shape == xgbtrain.shape)
assert(rfvalid.shape == xgbvalid.shape)
assert(rftest.shape == xgbtest.shape)

In [ ]:
mlptrain = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/mlp_train_pred.txt')
mlpvalid = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/mlp_val_pred.txt')
mlptest = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/mlp_test_pred.txt')
assert(mlptrain.shape == xgbtrain.shape)
assert(mlpvalid.shape == xgbvalid.shape)
assert(mlptest.shape == xgbtest.shape)

In [ ]:
logregtrain = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/LogisticRegressionTrain.txt')
logregvalid = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/LogisticRegressionVal.txt')
logregtest = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/LogisticRegressionTest.txt')
assert(logregtrain.shape == xgbtrain.shape)
assert(logregvalid.shape == xgbvalid.shape)
assert(logregtest.shape == xgbtest.shape)

In [ ]:
lgbmtrain = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/lightgbm_train_pred.txt')
lgbmvalid = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/lightgbm_val_pred.txt')
lgbmtest = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/lightgbm_test_pred.txt')
assert(lgbmtrain.shape == xgbtrain.shape)
assert(lgbmvalid.shape == xgbvalid.shape)
assert(lgbmtest.shape == xgbtest.shape)

In [ ]:
knntrain = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/KNNTrain.txt')
knnvalid = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/KNNVal.txt')
knntest = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/KNNTest.txt')
assert(knntrain.shape == xgbtrain.shape)
assert(knnvalid.shape == xgbvalid.shape)
assert(knntest.shape == xgbtest.shape)

In [ ]:
catboosttrain = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/cat_boost_train_pred.txt')
catboostvalid = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/cat_boost_val_pred.txt')
catboosttest = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/cat_boost_test_pred.txt')
assert(catboosttrain.shape == xgbtrain.shape)
assert(catboostvalid.shape == xgbvalid.shape)
assert(catboosttest.shape == xgbtest.shape)

In [ ]:
extratreestrain = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/ExtraTrees/ExtraTrees_100depth_train.txt')
extratreesvalid = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/ExtraTrees/ExtraTrees_100depth_val.txt')
extratreestest = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/ExtraTrees/ExtraTrees_100depth_test.txt')
assert(extratreestrain.shape == xgbtrain.shape)
assert(extratreesvalid.shape == xgbvalid.shape)
assert(extratreestest.shape == xgbtest.shape)

In [ ]:
xtrain_2ndlayer = np.hstack((x_train, xgbtrain, rftrain, mlptrain, logregtrain, lgbmtrain, knntrain, catboosttrain, extratreestrain))
xvalid_2ndlayer = np.hstack((x_valid, xgbvalid, rfvalid, mlpvalid, logregvalid, lgbmvalid, knnvalid, catboostvalid, extratreesvalid))
xtest_2ndlayer = np.hstack((testx, xgbtest, rftest, mlptest, logregtest, lgbmtest, knntest, catboosttest, extratreestest))
print(xtrain_2ndlayer.shape)
print(xvalid_2ndlayer.shape)
print(xtest_2ndlayer.shape)

(49502, 165)
(12376, 165)
(144368, 165)


In [ ]:
#save inputs to 2nd layer in google drive
np.savetxt('drive/My Drive/Applied Machine Learning Team Otto/xtrain_2ndlayer.txt',xtrain_2ndlayer)
np.savetxt('drive/My Drive/Applied Machine Learning Team Otto/xvalid_2ndlayer.txt',xvalid_2ndlayer)
np.savetxt('drive/My Drive/Applied Machine Learning Team Otto/xtest_2ndlayer.txt',xtest_2ndlayer)

In [ ]:
xtrain_2ndlayer = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/xtrain_2ndlayer.txt')
xvalid_2ndlayer = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/xvalid_2ndlayer.txt')
xtest_2ndlayer = np.loadtxt('drive/My Drive/Applied Machine Learning Team Otto/xtest_2ndlayer.txt')
# xtrain_2ndlayer = np.loadtxt('/content/drive/My Drive/Applied Machine Learning Team Otto/everyting2ndlayer_train.txt')
# xvalid_2ndlayer = np.loadtxt('/content/drive/My Drive/Applied Machine Learning Team Otto/everyting2ndlayer_valid.txt')
# xtest_2ndlayer = np.loadtxt('/content/drive/My Drive/Applied Machine Learning Team Otto/everyting2ndlayer_test.txt')

In [ ]:
#
# xgboost second layer
#
print(time.asctime())
# best result on 93cols + 8 models (x_2ndlayer, not everything2ndlayer_ above)
m = xgb.XGBClassifier(colsample_bylevel=0.25,
                      colsample_bytree=0.05, gamma=0,
                      learning_rate=0.05, max_depth=6,
                      min_child_weight=6, n_estimators=300, nthread=-1,
                      objective='multi:softprob', reg_alpha=0, reg_lambda=1,
                      scale_pos_weight=1, seed=0, silent=True, subsample=0.8,
                      tree_method='gpu_hist')

# work in progress model
# m = xgb.XGBClassifier(colsample_bylevel=0.1,
#                       colsample_bytree=0.05, gamma=0,
#                       learning_rate=0.05, max_depth=10,
#                       min_child_weight=6, n_estimators=300, nthread=-1,
#                       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
#                       scale_pos_weight=1, seed=0, silent=True, subsample=0.8,
#                       tree_method='gpu_hist')


m.fit(xtrain_2ndlayer, y_train, eval_metric='mlogloss', early_stopping_rounds=20, eval_set=[(xvalid_2ndlayer, y_valid)])
print(log_loss(y_valid, m.predict_proba(xvalid_2ndlayer)))
print(log_loss(y_train, m.predict_proba(xtrain_2ndlayer)))
print(time.asctime())

Sat Feb 22 18:01:01 2020
[0]	validation_0-mlogloss:2.11556
Will train until validation_0-mlogloss hasn't improved in 20 rounds.
[1]	validation_0-mlogloss:2.05602
[2]	validation_0-mlogloss:2.01796
[3]	validation_0-mlogloss:1.95281
[4]	validation_0-mlogloss:1.90648
[5]	validation_0-mlogloss:1.88089
[6]	validation_0-mlogloss:1.80737
[7]	validation_0-mlogloss:1.73455
[8]	validation_0-mlogloss:1.69169
[9]	validation_0-mlogloss:1.65235
[10]	validation_0-mlogloss:1.62223
[11]	validation_0-mlogloss:1.57472
[12]	validation_0-mlogloss:1.53567
[13]	validation_0-mlogloss:1.49204
[14]	validation_0-mlogloss:1.45981
[15]	validation_0-mlogloss:1.43105
[16]	validation_0-mlogloss:1.40368
[17]	validation_0-mlogloss:1.38314
[18]	validation_0-mlogloss:1.35343
[19]	validation_0-mlogloss:1.32331
[20]	validation_0-mlogloss:1.31214
[21]	validation_0-mlogloss:1.29174
[22]	validation_0-mlogloss:1.27832
[23]	validation_0-mlogloss:1.25662
[24]	validation_0-mlogloss:1.23134
[25]	validation_0-mlogloss:1.19646
[26]	v

In [ ]:
np.savetxt('drive/My Drive/Applied Machine Learning Team Otto/xgboost_2ndlayeroutput_train.txt',m.predict_proba(xtrain_2ndlayer))
np.savetxt('drive/My Drive/Applied Machine Learning Team Otto/xgboost_2ndlayeroutput_val.txt',m.predict_proba(xvalid_2ndlayer))
np.savetxt('drive/My Drive/Applied Machine Learning Team Otto/xgboost_2ndlayeroutput_test.txt',m.predict_proba(xtest_2ndlayer))

In [ ]:
#make sure m is defined correctly, and that testx has had the needed preprocessing steps applied
preds = m.predict_proba(xtest_2ndlayer)
#submission to kaggle, preds is output on test set
columns = ['Class_' + str(i + 1) for i in range(9)]
submission_df = pd.DataFrame(preds, columns=columns)
submission_df.index = submission_df.index + 1
submission_df.to_csv('submission.csv', sep=',', index_label='id')